In [1]:
from pyspark.sql import SparkSession, types, functions as F

In [2]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('events_etl')\
  .config("spark.driver.bindAddress", "127.0.0.1")\
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 17:21:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_nested = spark.read\
  .option('multiline', 'true')\
  .json('./data/events')

In [4]:
df_flat = df_nested.select(
  F.col('id').cast(types.StringType()),
  F.col('index').cast(types.IntegerType()),
  F.col('period').cast(types.IntegerType()),
  F.col('possession').cast(types.IntegerType()),
  F.col('timestamp'),
  F.col('minute').cast(types.IntegerType()),
  F.col('second').cast(types.IntegerType()),
  F.col('duration').cast(types.FloatType()),
  F.col('location'),
  # PLAY PATTERN
  F.col('play_pattern.id').cast(types.IntegerType()).alias('play_pattern_id'),
  F.col('play_pattern.name').cast(types.StringType()).alias('play_pattern'),
  F.col('type.id').cast(types.IntegerType()).alias('event_type_id'),
  F.col('type.name').cast(types.StringType()).alias('event_type_name'),
  # TEAM
  F.col('possession_team.id').cast(types.IntegerType()).alias('possession_team_id'),
  F.col('possession_team.name').cast(types.StringType()).alias('possession_team_name'),
  F.col('team.id').cast(types.IntegerType()).alias('event_team_id'),
  F.col('team.name').cast(types.StringType()).alias('event_team_name'),
  # PLAYER
  F.col('player.id').cast(types.IntegerType()).alias('player_id'),
  F.col('player.name').cast(types.StringType()).alias('player'),
  F.col('position.id').cast(types.IntegerType()).alias('player_position_id'),
  F.col('position.name').cast(types.StringType()).alias('player_position'),
  # PLAY TYPES
  F.col('50_50.outcome.id').cast(types.IntegerType()).alias('50_50_id'),
  F.col('50_50.outcome.name').cast(types.StringType()).alias('50_50_outcome'),
  F.col('bad_behaviour.card.id').cast(types.IntegerType()).alias('card_id'),
  F.col('bad_behaviour.card.name').cast(types.StringType()).alias('card'),
  F.col('ball_receipt.outcome.id').cast(types.IntegerType()).alias('ball_receipt_outcome_id'),
  F.col('ball_receipt.outcome.name').cast(types.StringType()).alias('ball_receipt_outcome'),
  F.col('ball_recovery.offensive').cast(types.BooleanType()).alias('ball_recovery_offensive'),
  F.col('ball_recovery.recovery_failure').cast(types.BooleanType()).alias('ball_recovery_failure'),
  F.col('block.deflection').cast(types.BooleanType()).alias('block_deflection'),
  F.col('block.offensive').cast(types.BooleanType()).alias('block_offensive'),
  F.col('block.save_block').cast(types.BooleanType()).alias('block_save'),
  F.col('carry.end_location').alias('carry_end_location'),
  F.col('clearance.aerial_won').cast(types.BooleanType()).alias('clearance_aerial_won'),
  F.col('clearance.body_part.id').cast(types.IntegerType()).alias('clearance_body_part_id'),
  F.col('clearance.body_part.name').cast(types.StringType()).alias('clearance_body_part'),
  F.col('clearance.head').cast(types.BooleanType()).alias('clearance_head'),
  F.col('clearance.left_foot').cast(types.BooleanType()).alias('clearance_left_foot'),
  F.col('clearance.right_foot').cast(types.BooleanType()).alias('clearance_right_foot'),
  F.col('clearance.other').cast(types.BooleanType()).alias('clearance_other'),
  F.col('counterpress').cast(types.BooleanType()),
  F.col('dribble.no_touch').cast(types.BooleanType()).alias('dribble_no_touch'),
  F.col('dribble.nutmeg').cast(types.BooleanType()).alias('dribble_nutmeg'),
  F.col('dribble.overrun').cast(types.BooleanType()).alias('dribble_overrun'),
  F.col('dribble.outcome.id').cast(types.IntegerType()).alias('dribble_id'),
  F.col('dribble.outcome.name').cast(types.StringType()).alias('dribble_name'),
  F.col('duel.outcome.id').cast(types.IntegerType()).alias('duel_outcome_id'),
  F.col('duel.outcome.name').cast(types.StringType()).alias('duel_outcome'),
  F.col('duel.type.id').cast(types.IntegerType()).alias('duel_type_id'),
  F.col('duel.type.name').cast(types.StringType()).alias('duel_type_name'),
  F.col('foul_committed.advantage').cast(types.BooleanType()).alias('foul_committed_advantage'),
  F.col('foul_committed.card.id').cast(types.IntegerType()).alias('foul_committed_card_id'),
  F.col('foul_committed.card.name').cast(types.StringType()).alias('foul_committed_card'),
  F.col('foul_committed.offensive').cast(types.BooleanType()).alias('foul_committed_offensive'),
  F.col('foul_committed.penalty').cast(types.BooleanType()).alias('foul_committed_penalty'),
  F.col('foul_committed.type.id').cast(types.IntegerType()).alias('foul_committed_type_id'),
  F.col('foul_committed.type.name').cast(types.StringType()).alias('foul_committed_type'),
  F.col('foul_won.advantage').cast(types.BooleanType()).alias('foul_won_advantage'),
  F.col('foul_won.defensive').cast(types.BooleanType()).alias('foul_won_defensive'),
  F.col('foul_won.penalty').cast(types.BooleanType()).alias('foul_won_penalty'),
  F.col('half_start.late_video_start').cast(types.BooleanType()).alias('half_late_start'),
  F.col('half_end.early_video_end').cast(types.BooleanType()).alias('half_early_end'),
  F.col('injury_stoppage.in_chain').cast(types.BooleanType()).alias('injury_stoppage_in_chain'),
  F.col('interception.outcome.id').cast(types.IntegerType()).alias('interception_outcome_id'),
  F.col('interception.outcome.name').cast(types.StringType()).alias('interception_outcome'),
  F.col('miscontrol.aerial_won').cast(types.BooleanType()).alias('miscontrol_aerial_won'),
  F.col('pass.aerial_won').cast(types.BooleanType()).alias('pass_aerial_won'),
  F.col('pass.angle').cast(types.FloatType()).alias('pass_angle'),
  F.col('pass.assisted_shot_id').cast(types.StringType()).alias('pass_assisted_shot_id'),
  F.col('pass.backheel').cast(types.BooleanType()).alias('backheel_pass'),
  F.col('pass.body_part.id').cast(types.IntegerType()).alias('pass_body_part_id'),
  F.col('pass.body_part.name').cast(types.StringType()).alias('pass_body_part'),
  F.col('pass.cross').cast(types.BooleanType()).alias('pass_cross'),
  F.col('pass.cut_back').cast(types.BooleanType()).alias('pass_cutback'),
  F.col('pass.deflected').cast(types.BooleanType()).alias('pass_deflected'),
  F.col('pass.end_location').alias('pass_endlocation'),
  F.col('pass.goal_assist').cast(types.BooleanType()).alias('pass_goal_assist'),
  F.col('pass.shot_assist').cast(types.BooleanType()).alias('pass_shot_assist'),
  F.col('pass.height.id').cast(types.IntegerType()).alias('pass_height_id'),
  F.col('pass.height.name').cast(types.StringType()).alias('pass_height'),
  F.col('pass.inswinging').cast(types.BooleanType()).alias('pass_inswinging'),
  F.col('pass.outswinging').cast(types.BooleanType()).alias('pass_outswinging'),
  F.col('pass.length').cast(types.FloatType()).alias('pass_length'),
  F.col('pass.miscommunication').cast(types.BooleanType()).alias('pass_miscommunication'),
  F.col('pass.no_touch').cast(types.BooleanType()).alias('pass_no_touch'),
  F.col('pass.outcome.id').cast(types.IntegerType()).alias('pass_outcome_id'),
  F.col('pass.outcome.name').cast(types.StringType()).alias('pass_outcome'),
  F.col('pass.recipient.id').cast(types.IntegerType()).alias('pass_recipient_id'),
  F.col('pass.recipient.name').cast(types.StringType()).alias('pass_recipient_name'),
  F.col('pass.straight').cast(types.BooleanType()).alias('pass_straight'),
  F.col('pass.switch').cast(types.BooleanType()).alias('pass_switch'),
  F.col('pass.through_ball').cast(types.BooleanType()).alias('pass_through_ball'),
  F.col('pass.technique.id').cast(types.IntegerType()).alias('pass_technique_id'),
  F.col('pass.technique.name').cast(types.StringType()).alias('pass_technique'),
  F.col('pass.type.id').cast(types.IntegerType()).alias('pass_type_id'),
  F.col('pass.type.name').cast(types.StringType()).alias('pass_type'),
  F.col('player_off.permanent').cast(types.BooleanType()).alias('player_off_permanent'),
  F.col('shot.aerial_won').cast(types.BooleanType()).alias('shot_aerial_won'),
  F.col('shot.body_part.id').cast(types.IntegerType()).alias('shot_body_part_id'),
  F.col('shot.body_part.name').cast(types.StringType()).alias('shot_body_part'),
  F.col('shot.deflected').cast(types.BooleanType()).alias('shot_deflected'),
  F.col('shot.end_location').alias('shot_endlocation'),
  F.col('shot.first_time').cast(types.BooleanType()).alias('shot_first_time'),
  F.col('shot.follows_dribble').cast(types.BooleanType()).alias('shot_follows_dribble'),
  F.col('shot.key_pass_id').cast(types.IntegerType()).alias('shot_key_pass_id'),
  F.col('shot.kick_off').cast(types.BooleanType()).alias('shot_kickoff'),
  F.col('shot.one_on_one').cast(types.BooleanType()).alias('shot_one_on_one'),
  F.col('shot.open_goal').cast(types.BooleanType()).alias('shot_open_goal'),
  F.col('shot.outcome.id').cast(types.IntegerType()).alias('shot_outcome_id'),
  F.col('shot.outcome.name').cast(types.StringType()).alias('shot_outcome'),
  F.col('shot.redirect').cast(types.BooleanType()).alias('shot_redirect'),
  F.col('shot.saved_off_target').cast(types.BooleanType()).alias('shot_saved_off_target'),
  F.col('shot.saved_to_post').cast(types.BooleanType()).alias('shot_saved_to_post'),
  F.col('shot.statsbomb_xg').cast(types.FloatType()).alias('shot_exg'),
  F.col('shot.technique.id').cast(types.IntegerType()).alias('shot_technique_id'),
  F.col('shot.technique.name').cast(types.StringType()).alias('shot_technique'),
  F.col('shot.type.id').cast(types.IntegerType()).alias('shot_type_id'),
  F.col('shot.type.name').cast(types.StringType()).alias('shot_type'),
  F.col('substitution.outcome.id').cast(types.IntegerType()).alias('sub_outcome_id'),
  F.col('substitution.outcome.name').cast(types.StringType()).alias('sub_outcome'),
  F.col('substitution.replacement.id').cast(types.IntegerType()).alias('sub_replacement_id'),
  F.col('substitution.replacement.name').cast(types.StringType()).alias('sub_replacement_name'),
  F.col('goalkeeper.body_part.id').cast(types.IntegerType()).alias('goalkeeper_body_part_id'),
  F.col('goalkeeper.body_part.name').cast(types.StringType()).alias('goalkeeper_body_part'),
  F.col('goalkeeper.end_location').alias('goalkeeper_end_location'),
  F.col('goalkeeper.lost_in_play').cast(types.BooleanType()).alias('goalkeeper_lost_in_play'),
  F.col('goalkeeper.lost_out').cast(types.BooleanType()).alias('goalkeeper_lost_out'),
  F.col('goalkeeper.outcome.id').cast(types.IntegerType()).alias('goalkeeper_outcome_id'),
  F.col('goalkeeper.outcome.name').cast(types.StringType()).alias('goalkeeper_outcome'),
  F.col('goalkeeper.penalty_saved_to_post').cast(types.BooleanType()).alias('goalkeeper_penalty_saved_to_post'),
  F.col('goalkeeper.position.id').cast(types.IntegerType()).alias('goalkeeper_position_id'),
  F.col('goalkeeper.position.name').cast(types.StringType()).alias('goalkeeper_position'),
  F.col('goalkeeper.punched_out').cast(types.BooleanType()).alias('goalkeeper_punched_out'),
  F.col('goalkeeper.saved_to_post').cast(types.BooleanType()).alias('goalkeeper_saved_to_post'),
  F.col('goalkeeper.shot_saved_off_target').cast(types.BooleanType()).alias('goalkeeper_shot_saved_off_target'),
  F.col('goalkeeper.shot_saved_to_post').cast(types.BooleanType()).alias('goalkeeper_shot_saved_to_post'),
  F.col('goalkeeper.success_in_play').cast(types.BooleanType()).alias('goalkeeper_success_in_play'),
  F.col('goalkeeper.success_out').cast(types.BooleanType()).alias('goalkeeper_success_out'),
  F.col('goalkeeper.technique.id').cast(types.IntegerType()).alias('goalkeeper_technique_id'),
  F.col('goalkeeper.technique.name').cast(types.StringType()).alias('goalkeeper_technique'),
  F.col('goalkeeper.type.id').cast(types.IntegerType()).alias('goalkeeper_type_id'),
  F.col('goalkeeper.type.name').cast(types.StringType()).alias('goalkeeper_type'),
  F.col('off_camera').cast(types.BooleanType()),
  F.col('out').cast(types.BooleanType()),
  F.col('under_pressure').cast(types.BooleanType())
)

In [5]:
df_flat.write.parquet('./data/pq/events')

25/04/16 17:22:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/04/16 17:22:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/16 17:22:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/16 17:22:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/16 17:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/16 17:22:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [6]:
spark.stop()